# 第05章：Tools & Tool Calling（工具与函数调用）

## 本章学习目标

1. 理解 Tools 的概念和作用
2. 掌握 @tool 装饰器创建自定义工具
3. 学会使用 bind_tools 绑定工具到模型
4. 理解 Tool Calling 的完整工作流程
5. 掌握 tool_calls 和 ToolMessage 的使用
6. 学会并行工具调用
7. 掌握 tool_choice 强制工具选择
8. 理解 Tools 与 Agent 的关系

## 核心概念

- **Tools（工具）**：AI能调用的函数，用于执行实际操作
- **@tool 装饰器**：将Python函数转换为工具
- **Tool Schema**：工具的输入输出定义（自动从类型注解生成）
- **bind_tools()**：将工具绑定到模型，让模型知道有哪些工具可用
- **tool_calls**：AI响应中包含的工具调用请求
- **ToolMessage**：将工具执行结果返回给模型
- **tool_choice**：控制模型如何选择工具（any、auto、指定工具）
- **parallel_tool_calls**：模型是否可以并行调用多个工具

## 为什么重要？

从第04章的结构化输出，我们让AI能返回**结构化数据**。
现在我们要让AI能**执行实际操作**——这就是工具的力量！

**Tools 的作用**：
- 让AI能查询数据库
- 让AI能搜索互联网
- 让AI能调用API
- 让AI能执行计算
- 让AI能操作外部系统

**Tools 是 Agent 的基础**：Agent 通过工具与世界交互！


In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,  # 工具调用建议使用低温度
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成")
print("Tools & Tool Calling 学习开始！")
print(f"模型: {model.model_name}")
print(f"支持工具调用: {hasattr(model, 'bind_tools')}")

环境配置完成
Tools & Tool Calling 学习开始！
模型: gpt-4.1-mini
支持工具调用: True


## 1. 创建第一个工具

### 什么是工具（Tool）？

在 LangChain 中，**工具（Tool）是一个带有明确输入输出定义的可调用函数**。

**工具包含两部分**：
1. **Schema（模式）**：工具的名称、描述和参数定义
2. **Function（函数）**：实际执行的Python函数

### 使用 @tool 装饰器

最简单的创建工具的方式是使用 `@tool` 装饰器：

```python
from langchain.tools import tool

@tool
def my_tool(arg1: str, arg2: int) -> str:
    """工具描述（模型会读取这个docstring）
    
    Args:
        arg1: 参数1的描述
        arg2: 参数2的描述
    """
    return f"结果: {arg1}, {arg2}"
```

**重要规则**：
- **类型注解是必需的**：定义工具的输入schema
- **docstring很重要**：模型根据它决定何时使用这个工具
- **参数描述要清晰**：帮助模型正确传参
- **返回值类型**：通常返回字符串（会传回给模型）


In [2]:
# 创建第一个工具
print("【创建第一个简单工具】")

@tool
def get_weather(location: str) -> str:
    """获取指定地点的天气信息
    
    Args:
        location: 城市名称，例如：北京、上海、深圳
    """
    # 这里是模拟的天气数据
    weather_data = {
        "北京": "晴天，15°C，空气质量良好",
        "上海": "多云，18°C，有轻度雾霾",
        "深圳": "阴天，22°C，湿度较大",
        "广州": "小雨，20°C，建议带伞",
    }
    
    result = weather_data.get(location, f"抱歉，暂无{location}的天气数据")
    return f"{location}的天气：{result}"


# 查看工具的属性
print("工具名称:", get_weather.name)
print("工具描述:", get_weather.description)
print("工具参数schema:", get_weather.args)
print()

# 直接调用工具（像普通函数一样）
print("【直接调用工具】")
result = get_weather.invoke({"location": "北京"})
print(f"结果1: {result}")
print()

# 也可以用普通的函数调用方式
result2 = get_weather.run("上海")
print(f"结果2: {result2}")

【创建第一个简单工具】
工具名称: get_weather
工具描述: 获取指定地点的天气信息

    Args:
        location: 城市名称，例如：北京、上海、深圳
工具参数schema: {'location': {'title': 'Location', 'type': 'string'}}

【直接调用工具】
结果1: 北京的天气：晴天，15°C，空气质量良好

结果2: 上海的天气：多云，18°C，有轻度雾霾


## 2. bind_tools - 将工具绑定到模型

### Tool Calling 工作流程

要让模型能够调用工具，需要经历以下步骤：

```
1. 创建工具（@tool装饰器）
2. 绑定工具到模型（bind_tools）
3. 用户提问
4. 模型决定是否需要调用工具
   - 如果需要：返回 tool_calls（工具调用请求）
   - 如果不需要：直接返回答案
5. 执行工具调用，获得结果
6. 将结果包装为 ToolMessage 返回给模型
7. 模型基于工具结果生成最终答案
```

### bind_tools() 方法

`bind_tools()` 将工具列表绑定到模型，让模型知道有哪些工具可用：

```python
tools = [tool1, tool2, tool3]
model_with_tools = model.bind_tools(tools)
```

**绑定后的模型有什么变化？**
- 模型的 **system prompt 会自动包含工具的schema**
- 模型会根据用户输入**决定是否调用工具**
- 模型的响应中可能包含 **tool_calls 字段**

### AIMessage 中的 tool_calls

当模型决定调用工具时，响应中会包含 `tool_calls`：

```python
response.tool_calls = [
    {
        "name": "get_weather",      # 工具名称
        "args": {"location": "北京"}, # 参数
        "id": "call_abc123"          # 唯一标识
    }
]
```


In [3]:
# 将工具绑定到模型
print("【bind_tools - 绑定工具到模型】")

# 创建带工具的模型
model_with_tools = model.bind_tools([get_weather])

# 测试1：需要调用工具的问题
print("【测试1：询问天气（需要工具）】")
question1 = "北京今天天气怎么样？"
response1 = model_with_tools.invoke(question1)

print(f"问题: {question1}")
print(f"响应类型: {type(response1)}")
print(f"响应内容: {response1.content}")
print(f"tool_calls: {response1.tool_calls}")
print()

if response1.tool_calls:
    print("模型决定调用工具！")
    for tool_call in response1.tool_calls:
        print(f"  工具名称: {tool_call['name']}")
        print(f"  工具参数: {tool_call['args']}")
        print(f"  调用ID: {tool_call['id']}")
print()

# 测试2：不需要工具的问题
print("【测试2：普通问题（不需要工具）】")
question2 = "你好，请介绍一下自己"
response2 = model_with_tools.invoke(question2)

print(f"问题: {question2}")
print(f"响应内容: {response2.content}")
print(f"tool_calls: {response2.tool_calls}")
print()

if not response2.tool_calls:
    print("模型决定不调用工具，直接回答")
    
print("\n【总结】")
print("- 模型会根据问题智能判断是否需要工具")
print("- tool_calls 为空列表 [] 表示不需要工具")
print("- tool_calls 有内容表示需要调用工具")

【bind_tools - 绑定工具到模型】
【测试1：询问天气（需要工具）】
问题: 北京今天天气怎么样？
响应类型: <class 'langchain_core.messages.ai.AIMessage'>
响应内容: 
tool_calls: [{'name': 'get_weather', 'args': {'location': '北京'}, 'id': 'call_uq708dCd9dPWU2i5oKBbAoRQ', 'type': 'tool_call'}]

模型决定调用工具！
  工具名称: get_weather
  工具参数: {'location': '北京'}
  调用ID: call_uq708dCd9dPWU2i5oKBbAoRQ

【测试2：普通问题（不需要工具）】
问题: 你好，请介绍一下自己
响应内容: 你好！我是ChatGPT，一个由OpenAI开发的人工智能语言模型。我可以帮助你回答问题、提供建议、撰写文本、翻译语言、生成创意内容等等。如果你有任何问题或需要帮助，随时告诉我！你有什么想了解或需要帮助的吗？
tool_calls: []

模型决定不调用工具，直接回答

【总结】
- 模型会根据问题智能判断是否需要工具
- tool_calls 为空列表 [] 表示不需要工具
- tool_calls 有内容表示需要调用工具


## 3. 完整的工具调用循环

### 工具调用的完整流程

当模型返回 `tool_calls` 后，我们需要：
1. **执行工具**：调用实际的工具函数
2. **创建 ToolMessage**：将工具结果包装为消息
3. **继续对话**：将 ToolMessage 发送回模型
4. **获得最终答案**：模型基于工具结果生成回答

### ToolMessage 结构

```python
ToolMessage(
    content="工具执行结果（字符串）",
    tool_call_id="call_abc123",  # 必须与 tool_call 的 id 匹配
    name="工具名称"
)
```

**关键点**：
- `content`：工具执行的结果（字符串格式）
- `tool_call_id`：必须与原始 `tool_call` 的 `id` 匹配
- `name`：工具的名称

### 为什么需要完整循环？

模型返回的 `tool_calls` **只是请求**，不是结果！
我们需要：
1. 实际执行工具
2. 把结果告诉模型
3. 模型才能基于真实结果回答用户


In [4]:
# 完整的工具调用循环
print("【完整的Tool Calling循环演示】")
print()

# 步骤1：用户提问
user_question = "深圳和广州今天的天气分别是什么？"
print(f"步骤1 - 用户提问: {user_question}")
print()

# 步骤2：模型决定调用工具
print("步骤2 - 模型处理...")
messages = [HumanMessage(content=user_question)]
ai_response = model_with_tools.invoke(messages)

print(f"模型响应content: {ai_response.content}")
print(f"模型是否调用工具: {'是' if ai_response.tool_calls else '否'}")
print(f"工具调用数量: {len(ai_response.tool_calls)}")
print()

# 步骤3：执行工具调用
if ai_response.tool_calls:
    print("步骤3 - 执行工具调用...")
    
    # 保存AI的响应
    messages.append(ai_response)
    
    # 创建工具字典，方便查找
    tools_dict = {get_weather.name: get_weather}
    
    # 执行每个工具调用
    for tool_call in ai_response.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]
        
        print(f"  调用工具: {tool_name}")
        print(f"  参数: {tool_args}")
        
        # 获取工具并执行
        tool = tools_dict[tool_name]
        tool_result = tool.invoke(tool_args)
        
        print(f"  结果: {tool_result}")
        print()
        
        # 创建 ToolMessage
        tool_message = ToolMessage(
            content=tool_result,
            tool_call_id=tool_id,
            name=tool_name
        )
        
        # 将工具结果添加到消息列表
        messages.append(tool_message)
    
    # 步骤4：将工具结果发送回模型，获得最终答案
    print("步骤4 - 模型基于工具结果生成最终答案...")
    final_response = model_with_tools.invoke(messages)
    
    print(f"最终答案: {final_response.content}")
    print()
    
print("=" * 60)
print("【完整的消息历史】")
for i, msg in enumerate(messages + [final_response], 1):
    msg_type = type(msg).__name__
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"{i}. {msg_type}: [包含 {len(msg.tool_calls)} 个工具调用]")
    else:
        content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
        print(f"{i}. {msg_type}: {content}")

【完整的Tool Calling循环演示】

步骤1 - 用户提问: 深圳和广州今天的天气分别是什么？

步骤2 - 模型处理...
模型响应content: 
模型是否调用工具: 是
工具调用数量: 2

步骤3 - 执行工具调用...
  调用工具: get_weather
  参数: {'location': '深圳'}
  结果: 深圳的天气：阴天，22°C，湿度较大

  调用工具: get_weather
  参数: {'location': '广州'}
  结果: 广州的天气：小雨，20°C，建议带伞

步骤4 - 模型基于工具结果生成最终答案...
最终答案: 深圳今天的天气是阴天，气温22°C，湿度较大。广州今天有小雨，气温20°C，建议带伞。

【完整的消息历史】
1. HumanMessage: 深圳和广州今天的天气分别是什么？
2. AIMessage: [包含 2 个工具调用]
3. ToolMessage: 深圳的天气：阴天，22°C，湿度较大
4. ToolMessage: 广州的天气：小雨，20°C，建议带伞
5. AIMessage: 深圳今天的天气是阴天，气温22°C，湿度较大。广州今天有小雨，气温20°C，建议带伞。


## 4. 多工具绑定与选择

### 绑定多个工具

实际应用中，我们通常会给模型绑定多个工具，让模型根据需求选择：

```python
model_with_tools = model.bind_tools([tool1, tool2, tool3, tool4])
```

**模型如何选择工具？**
- 模型会根据**工具的名称和描述**理解工具的用途
- 模型会根据**用户的问题**判断需要哪个工具
- 模型可以选择**不调用任何工具**
- 模型可以**连续调用多个工具**

### 创建不同类型的工具

让我们创建几个不同类型的工具来演示：


In [6]:
from langchain_core.tools import tool

# 创建多个不同类型的工具
print("【创建多个工具】")

# 工具1：计算器
@tool
def calculator(expression: str) -> str:
    """执行数学计算
    
    Args:
        expression: 数学表达式，例如：2+3*4、10/2、sqrt(16)
    """
    try:
        # 安全的数学计算
        import math
        allowed_names = {
            'abs': abs, 'round': round, 'pow': pow,
            'sqrt': math.sqrt, 'pi': math.pi, 'e': math.e
        }
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"计算结果：{expression} = {result}"
    except Exception as e:
        return f"计算错误：{str(e)}"


# 工具2：查询用户信息
@tool
def get_user_info(user_id: int) -> str:
    """查询用户信息
    
    Args:
        user_id: 用户ID（整数）
    """
    # 模拟用户数据库
    users = {
        1001: {"name": "张三", "age": 28, "city": "北京", "vip": True},
        1002: {"name": "李四", "age": 35, "city": "上海", "vip": False},
        1003: {"name": "王五", "age": 42, "city": "深圳", "vip": True},
    }
    
    user = users.get(user_id)
    if user:
        vip_status = "VIP会员" if user["vip"] else "普通用户"
        return f"用户{user_id}：{user['name']}，{user['age']}岁，来自{user['city']}，{vip_status}"
    else:
        return f"未找到用户ID: {user_id}"


# 工具3：翻译工具
@tool
def translate(text: str, target_language: str = "英语") -> str:
    """将文本翻译为目标语言
    
    Args:
        text: 要翻译的文本
        target_language: 目标语言，可选：英语、日语、法语、德语
    """
    # 这里是模拟的翻译
    translations = {
        "你好": {"英语": "Hello", "日语": "こんにちは", "法语": "Bonjour", "德语": "Hallo"},
        "谢谢": {"英语": "Thank you", "日语": "ありがとう", "法语": "Merci", "德语": "Danke"},
        "再见": {"英语": "Goodbye", "日语": "さようなら", "法语": "Au revoir", "德语": "Auf Wiedersehen"},
    }
    
    if text in translations and target_language in translations[text]:
        result = translations[text][target_language]
        return f"'{text}' 翻译为{target_language}：{result}"
    else:
        return f"抱歉，暂不支持'{text}'翻译为{target_language}"


print("工具1: calculator - 数学计算")
print("工具2: get_user_info - 查询用户信息")
print("工具3: translate - 翻译文本")
print("工具4: get_weather - 查询天气")
print()

# 绑定所有工具
all_tools = [calculator, get_user_info, translate, get_weather]
model_with_multiple_tools = model.bind_tools(all_tools)

print(f"已绑定 {len(all_tools)} 个工具到模型")

【创建多个工具】
工具1: calculator - 数学计算
工具2: get_user_info - 查询用户信息
工具3: translate - 翻译文本
工具4: get_weather - 查询天气

已绑定 4 个工具到模型


In [7]:
# 测试模型的工具选择能力
print("【测试模型如何选择工具】")
print()

test_questions = [
    "计算 15 * 8 + 32 的结果",
    "查询用户ID 1002的信息",
    "把'你好'翻译成日语",
    "上海的天气怎么样？",
    "你是谁？",  # 不需要工具
]

# 创建工具字典，方便执行
tools_dict = {t.name: t for t in all_tools}

for i, question in enumerate(test_questions, 1):
    print(f"【测试 {i}】问题: {question}")
    
    response = model_with_multiple_tools.invoke(question)
    
    if response.tool_calls:
        print(f"决策: 需要调用工具")
        print(f"工具调用数: {len(response.tool_calls)}")
        
        for tc in response.tool_calls:
            tool_name = tc["name"]
            tool_args = tc["args"]
            print(f"  - 工具: {tool_name}")
            print(f"    参数: {tool_args}")
            
            # 执行工具
            tool = tools_dict[tool_name]
            result = tool.invoke(tool_args)
            print(f"    结果: {result}")
    else:
        print(f"决策: 不需要工具")
        print(f"直接回答: {response.content}")
    
    print("-" * 60)
    print()

print("【总结】")
print("- 模型能根据问题自动选择正确的工具")
print("- 模型知道什么时候需要工具，什么时候不需要")
print("- 工具的描述（docstring）对模型的决策非常重要")

【测试模型如何选择工具】

【测试 1】问题: 计算 15 * 8 + 32 的结果
决策: 需要调用工具
工具调用数: 1
  - 工具: calculator
    参数: {'expression': '15 * 8 + 32'}
    结果: 计算结果：15 * 8 + 32 = 152
------------------------------------------------------------

【测试 2】问题: 查询用户ID 1002的信息
决策: 需要调用工具
工具调用数: 1
  - 工具: get_user_info
    参数: {'user_id': 1002}
    结果: 用户1002：李四，35岁，来自上海，普通用户
------------------------------------------------------------

【测试 3】问题: 把'你好'翻译成日语
决策: 需要调用工具
工具调用数: 1
  - 工具: translate
    参数: {'text': '你好', 'target_language': '日语'}
    结果: '你好' 翻译为日语：こんにちは
------------------------------------------------------------

【测试 4】问题: 上海的天气怎么样？
决策: 需要调用工具
工具调用数: 1
  - 工具: get_weather
    参数: {'location': '上海'}
    结果: 上海的天气：多云，18°C，有轻度雾霾
------------------------------------------------------------

【测试 5】问题: 你是谁？
决策: 不需要工具
直接回答: 我是一个由人工智能驱动的助手，可以帮助你回答问题、提供信息、执行各种任务。你有什么需要帮助的吗？
------------------------------------------------------------

【总结】
- 模型能根据问题自动选择正确的工具
- 模型知道什么时候需要工具，什么时候不需要
- 工具的描述（docstring）对模型的决策非常重要


## 5. 并行工具调用（Parallel Tool Calls）

### 什么是并行工具调用？

当多个工具调用**相互独立**时，模型可以**同时请求多个工具调用**，提高效率。

**示例**：
- 用户问："北京和上海的天气分别是什么？"
- 模型可以同时调用：
  - `get_weather("北京")`
  - `get_weather("上海")`

### 并行调用的特点

**优点**：
- 减少往返次数（只需一次模型调用）
- 提高效率
- 适合独立的信息查询

**默认行为**：
- 大多数支持工具调用的模型**默认启用**并行调用
- OpenAI 和 Anthropic 支持并行工具调用

### 控制并行行为

如果需要禁用并行调用：

```python
model_with_tools = model.bind_tools(
    tools,
    parallel_tool_calls=False  # 禁用并行调用
)
```


In [8]:
# 并行工具调用演示
print("【并行工具调用演示】")
print()

# 测试问题：需要查询多个独立信息
parallel_question = "帮我完成以下任务：1) 计算 25*4 2) 查询用户1001的信息 3) 北京的天气"

print(f"问题: {parallel_question}")
print()

response = model_with_multiple_tools.invoke(parallel_question)

print(f"模型响应:")
print(f"- content: {response.content}")
print(f"- 工具调用数: {len(response.tool_calls)}")
print()

if response.tool_calls:
    print("【工具调用详情】")
    for i, tc in enumerate(response.tool_calls, 1):
        print(f"{i}. 工具: {tc['name']}")
        print(f"   参数: {tc['args']}")
        print(f"   ID: {tc['id']}")
        print()
    
    print("【执行所有工具调用】")
    # 执行所有工具
    for tc in response.tool_calls:
        tool = tools_dict[tc["name"]]
        result = tool.invoke(tc["args"])
        print(f"- {tc['name']}: {result}")

print()
print("【并行调用的价值】")
print("- 一次模型调用就能获得多个工具调用请求")
print("- 节省了多次往返的时间")
print("- 适合多个独立任务的场景")
print()

# 对比：禁用并行调用
print("【对比：禁用并行调用】")
model_sequential = model.bind_tools(all_tools, parallel_tool_calls=False)
response_seq = model_sequential.invoke("北京和上海的天气分别是什么？")

print(f"禁用并行后，工具调用数: {len(response_seq.tool_calls)}")
print("（通常会减少，或者需要多次往返）")

【并行工具调用演示】

问题: 帮我完成以下任务：1) 计算 25*4 2) 查询用户1001的信息 3) 北京的天气

模型响应:
- content: 
- 工具调用数: 3

【工具调用详情】
1. 工具: calculator
   参数: {'expression': '25*4'}
   ID: call_xw51xdAl7tb0nWPCcv5FcyLq

2. 工具: get_user_info
   参数: {'user_id': 1001}
   ID: call_1D8tUC2AYEqBVpZ0iIefa92f

3. 工具: get_weather
   参数: {'location': '北京'}
   ID: call_kkBbrTRyDdRBZT2JVjAlnZX9

【执行所有工具调用】
- calculator: 计算结果：25*4 = 100
- get_user_info: 用户1001：张三，28岁，来自北京，VIP会员
- get_weather: 北京的天气：晴天，15°C，空气质量良好

【并行调用的价值】
- 一次模型调用就能获得多个工具调用请求
- 节省了多次往返的时间
- 适合多个独立任务的场景

【对比：禁用并行调用】
禁用并行后，工具调用数: 2
（通常会减少，或者需要多次往返）


## 6. tool_choice - 强制工具选择

### tool_choice 参数

默认情况下，模型可以**自由选择**是否调用工具以及调用哪个工具。
但有时我们需要**强制模型调用工具**。

### tool_choice 的取值

```python
# 1. "auto" - 默认行为，模型自动决定
model.bind_tools(tools, tool_choice="auto")

# 2. "any" - 强制模型必须调用某个工具（任意一个）
model.bind_tools(tools, tool_choice="any")

# 3. 指定工具名 - 强制调用特定工具
model.bind_tools(tools, tool_choice="get_weather")
```

### 使用场景

**"any" 的使用场景**：
- 确保模型一定会使用工具
- 避免模型直接回答而不调用工具
- 适合结构化数据提取场景

**指定工具名的场景**：
- 强制使用某个特定工具
- 用于测试和调试
- 构建固定流程的应用


In [9]:
from langchain_core.tools import tool

# tool_choice 参数演示
print("【tool_choice 参数演示】")
print()

test_question = "今天天气怎么样？"  # 模糊的问题

# 测试1：auto - 默认行为
print("【测试1：tool_choice='auto'（默认）】")
model_auto = model.bind_tools([get_weather], tool_choice="auto")
response_auto = model_auto.invoke(test_question)

print(f"问题: {test_question}")
print(f"是否调用工具: {'是' if response_auto.tool_calls else '否'}")
if not response_auto.tool_calls:
    print(f"直接回答: {response_auto.content[:100]}")
print()

# 测试2：any - 强制使用工具
print("【测试2：tool_choice='any'（强制使用工具）】")
model_any = model.bind_tools([get_weather], tool_choice="any")
response_any = model_any.invoke(test_question)

print(f"问题: {test_question}")
print(f"是否调用工具: {'是' if response_any.tool_calls else '否'}")
if response_any.tool_calls:
    print(f"工具: {response_any.tool_calls[0]['name']}")
    print(f"参数: {response_any.tool_calls[0]['args']}")
print()

# 测试3：指定特定工具
print("【测试3：tool_choice='calculator'（指定工具）】")
model_specific = model.bind_tools(all_tools, tool_choice="calculator")
response_specific = model_specific.invoke("帮我算一下 10 + 20")

print(f"问题: 帮我算一下 10 + 20")
print(f"调用的工具: {response_specific.tool_calls[0]['name'] if response_specific.tool_calls else '无'}")
if response_specific.tool_calls:
    print(f"参数: {response_specific.tool_calls[0]['args']}")
print()

print("【总结】")
print("- 'auto': 模型自由决定，可能不调用工具")
print("- 'any': 强制模型必须调用某个工具")
print("- 指定工具名: 强制调用特定工具")
print()

# 实际应用示例：结构化数据提取
print("【实际应用：强制工具调用做结构化提取】")

@tool  
def extract_contact_info(name: str, phone: str, email: str) -> str:
    """从文本中提取联系人信息
    
    Args:
        name: 姓名
        phone: 电话号码
        email: 邮箱地址
    """
    return f"已提取：姓名={name}, 电话={phone}, 邮箱={email}"

model_extractor = model.bind_tools([extract_contact_info], tool_choice="any")

contact_text = "我叫张伟，电话是13800138000，邮箱是zhangwei@example.com"
response_extract = model_extractor.invoke(f"从以下文本提取联系人信息：{contact_text}")

print(f"输入文本: {contact_text}")
if response_extract.tool_calls:
    args = response_extract.tool_calls[0]['args']
    print(f"提取结果: {args}")
    print()
    print("通过强制工具调用，我们确保了结构化输出！")

【tool_choice 参数演示】

【测试1：tool_choice='auto'（默认）】
问题: 今天天气怎么样？
是否调用工具: 否
直接回答: 请问您想查询哪个城市的天气？

【测试2：tool_choice='any'（强制使用工具）】
问题: 今天天气怎么样？
是否调用工具: 是
工具: get_weather
参数: {'location': '北京'}

【测试3：tool_choice='calculator'（指定工具）】
问题: 帮我算一下 10 + 20
调用的工具: calculator
参数: {'expression': '10+20'}

【总结】
- 'auto': 模型自由决定，可能不调用工具
- 'any': 强制模型必须调用某个工具
- 指定工具名: 强制调用特定工具

【实际应用：强制工具调用做结构化提取】
输入文本: 我叫张伟，电话是13800138000，邮箱是zhangwei@example.com
提取结果: {'name': '张伟', 'phone': '13800138000', 'email': 'zhangwei@example.com'}

通过强制工具调用，我们确保了结构化输出！


## 7. 实战项目：智能助手系统

### 项目概述

构建一个**多功能智能助手**，能够：
- 查询天气信息
- 进行数学计算
- 查询和管理用户信息
- 执行文本搜索
- 多轮对话，记忆上下文

**技术要点**：
- 多工具绑定和选择
- 完整的工具调用循环
- 消息历史管理
- 错误处理
- 多轮对话支持

### 系统架构

```
用户输入 → 模型分析 → 决定工具 → 执行工具 → 返回结果 → 生成回答 → 用户
            ↑                                                      ↓
            └──────────────────── 继续对话 ←────────────────────────┘
```


In [10]:
# 实战项目：智能助手系统
print("【智能助手系统实战项目】")
print()

# 1. 准备工具集
@tool
def search_knowledge(query: str) -> str:
    """搜索知识库
    
    Args:
        query: 搜索关键词
    """
    # 模拟知识库
    knowledge_base = {
        "LangChain": "LangChain是一个用于构建基于语言模型应用的框架，支持工具调用、RAG、Agent等功能。",
        "Python": "Python是一种高级编程语言，以其简洁的语法和强大的库生态系统而闻名。",
        "AI": "人工智能(AI)是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。",
    }
    
    for key, value in knowledge_base.items():
        if query.lower() in key.lower():
            return f"找到相关信息：{value}"
    
    return f"未找到关于'{query}'的信息"


# 工具列表
assistant_tools = [
    get_weather,
    calculator, 
    get_user_info,
    search_knowledge
]

# 2. 创建智能助手类
class SmartAssistant:
    """智能助手类"""
    
    def __init__(self, model, tools):
        self.model = model.bind_tools(tools)
        self.tools_dict = {t.name: t for t in tools}
        self.conversation_history = []
        
    def process_message(self, user_input: str) -> str:
        """处理用户消息"""
        # 添加用户消息到历史
        self.conversation_history.append(HumanMessage(content=user_input))
        
        # 模型处理
        response = self.model.invoke(self.conversation_history)
        self.conversation_history.append(response)
        
        # 如果需要工具调用
        if response.tool_calls:
            # 执行工具调用
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]
                tool_id = tool_call["id"]
                
                # 执行工具
                tool = self.tools_dict.get(tool_name)
                if tool:
                    try:
                        result = tool.invoke(tool_args)
                    except Exception as e:
                        result = f"工具执行错误: {str(e)}"
                else:
                    result = f"未找到工具: {tool_name}"
                
                # 创建工具消息
                tool_message = ToolMessage(
                    content=result,
                    tool_call_id=tool_id,
                    name=tool_name
                )
                self.conversation_history.append(tool_message)
            
            # 基于工具结果生成最终回答
            final_response = self.model.invoke(self.conversation_history)
            self.conversation_history.append(final_response)
            return final_response.content
        else:
            # 直接返回
            return response.content
    
    def reset(self):
        """重置对话历史"""
        self.conversation_history = []


# 3. 创建助手实例
assistant = SmartAssistant(model, assistant_tools)

print("智能助手已启动！")
print("可用工具:", [t.name for t in assistant_tools])
print()

# 4. 测试多轮对话
test_conversations = [
    "你好，请介绍一下自己",
    "北京今天天气怎么样？",
    "帮我计算一下 100 * 50 + 2000",
    "查询用户1001的信息",
    "告诉我LangChain是什么",
    "谢谢你的帮助！",
]

print("=" * 60)
print("【多轮对话测试】")
print("=" * 60)
print()

for i, user_msg in enumerate(test_conversations, 1):
    print(f"回合 {i}")
    print(f"👤 用户: {user_msg}")
    
    response = assistant.process_message(user_msg)
    
    print(f"🤖 助手: {response}")
    print()
    print("-" * 60)
    print()

print("【对话统计】")
print(f"总消息数: {len(assistant.conversation_history)}")
print(f"用户消息: {sum(1 for m in assistant.conversation_history if isinstance(m, HumanMessage))}")
print(f"AI消息: {sum(1 for m in assistant.conversation_history if isinstance(m, AIMessage))}")
print(f"工具消息: {sum(1 for m in assistant.conversation_history if isinstance(m, ToolMessage))}")

【智能助手系统实战项目】

智能助手已启动！
可用工具: ['get_weather', 'calculator', 'get_user_info', 'search_knowledge']

【多轮对话测试】

回合 1
👤 用户: 你好，请介绍一下自己
🤖 助手: 你好！我是一个由人工智能驱动的语言模型，旨在帮助你解答问题、提供信息、协助完成各种任务。无论是学习、工作还是生活中的疑问，我都可以为你提供支持。你有什么需要帮助的吗？

------------------------------------------------------------

回合 2
👤 用户: 北京今天天气怎么样？
🤖 助手: 北京今天天气晴朗，气温大约15摄氏度，空气质量良好。需要我帮你查询其他城市的天气吗？

------------------------------------------------------------

回合 3
👤 用户: 帮我计算一下 100 * 50 + 2000
🤖 助手: 计算结果是7000。还有其他需要帮忙计算的吗？

------------------------------------------------------------

回合 4
👤 用户: 查询用户1001的信息
🤖 助手: 用户1001的信息是：张三，28岁，来自北京，是VIP会员。还有其他需要查询的吗？

------------------------------------------------------------

回合 5
👤 用户: 告诉我LangChain是什么
🤖 助手: LangChain是一个用于构建基于语言模型应用的框架，支持工具调用、检索增强生成（RAG）、智能代理（Agent）等功能。它帮助开发者更方便地利用语言模型构建复杂的应用。你还想了解更多细节吗？

------------------------------------------------------------

回合 6
👤 用户: 谢谢你的帮助！
🤖 助手: 不客气！如果以后还有任何问题，随时来找我。祝你有美好的一天！

------------------------------------------------------------

【对话统计】
总消息

## 本章总结

### 核心知识点回顾

**Tools 的概念**
- 工具是带有明确输入输出定义的可调用函数
- 工具让AI能够与外部世界交互
- 工具是Agent的基础能力

**创建工具**
- 使用 @tool 装饰器快速创建工具
- 类型注解是必需的
- docstring很重要
- 返回值通常是字符串

**bind_tools 绑定工具**
- 将工具列表绑定到模型
- 模型会根据用户输入决定是否调用工具

**Tool Calling 工作流程**
1. 用户提问
2. 模型返回 tool_calls
3. 执行工具获得结果
4. 创建 ToolMessage
5. 将结果发送回模型
6. 模型生成最终答案

**并行工具调用**
- 模型可同时请求多个独立的工具调用
- 减少往返次数，提高效率

**tool_choice 参数**
- "auto": 模型自由决定
- "any": 强制使用工具
- 指定工具名：强制调用特定工具
